# Make Cube Network

In [36]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O

In [39]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'network_wrangler', 'examples', 'stpaul')
output_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'network_wrangler', 'examples', 'stpaul', 'cube')

## Read Roadway and Transit Networks

In [40]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2020-07-21 10:10:42, INFO: Reading from following files:
-/Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/link.json
-/Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/node.geojson
-/Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/shape.geojson.
2020-07-21 10:10:47, INFO: Read 66253 links from /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/link.json
2020-07-21 10:10:47, INFO: Read 17159 nodes from /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/node.geojson
2020-07-21 10:10:47, INFO: Read 66253 shapes from /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/shape.geojson


In [41]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2020-07-21 10:10:58, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul
2020-07-21 10:10:58, INFO: Removing calendar.txt from transit network config because file not found
2020-07-21 10:10:58, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-07-21 10:10:58, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-07-21 10:10:58, INFO: Removing fare_rules.txt from transit network config because file not found
2020-07-21 10:10:58, INFO: Removing transfers.txt from transit network config because file not found
2020-07-21 10:10:58, INFO: Removing feed_info.txt from transit network config because file not found


## Make Travel Model Networks

In [42]:
model_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(roadway_network_object = roadway_net)
model_roadway_net.roadway_standard_to_met_council_network()

2020-07-21 10:11:09, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2020-07-21 10:11:10, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-07-21 10:11:10, INFO: Didn't detect managed lanes in network.
2020-07-21 10:11:10, INFO: Creating calculated roadway variables.
2020-07-21 10:11:10, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:374: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-07-21 10:11:17, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-07-21 10:11:17, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-07-21 10:11:24, INFO: Finished Calculating county variable: county
2020-07-21 10:11:24, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-07-21 10:11:24, INFO: Finished calculating centroid connector variable: centroidconnect
2020-07-21 10:11:24, INFO: Calculating MPO as roadway network variable: mpo
2020-07-21 10:11:24, INFO: Finished calculating MPO variable: mpo
2020-07-21 10:11:24, INFO: Calculating Assignment Group as network variable: assign_group
2020-07-21 10:11:24, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-07-21 10:11:53, INFO: Finished calculating assignment group variable: assign_group
2020-07-21 10:11:54, INFO: Calculating Roadway Class
2020-07-21 10:11:54, INFO: Finished calculating roadway class variable: roadway_class
2020-07-21 10:11:54, INFO: Adding Counts
2020-07-21 10:11:54, INFO: Adding Variable AADT using Shared Streets Reference from /Users/wsp/Do

## Write Networks to Disk as SHP and TXT

In [43]:
model_roadway_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
)

2020-07-21 10:12:03, INFO: Writing Network as Shapefile
2020-07-21 10:12:03, INFO: Renaming DBF Node Variables
2020-07-21 10:12:03, INFO: Renaming variables so that they are DBF-safe
2020-07-21 10:12:03, INFO: Renaming DBF Link Variables
2020-07-21 10:12:03, INFO: Renaming variables so that they are DBF-safe
2020-07-21 10:12:03, INFO: Writing Node Shapes:
 - /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/cube/nodes.shp
2020-07-21 10:12:05, INFO: Writing Link Shapes:
 - /Users/wsp/Documents/GitHub/network_wrangler/examples/stpaul/cube/links.shp
2020-07-21 10:12:09, INFO: Writing Network Data to CSVs:
 - /Users/wsp/Documents/GitHub/Lasso/tests/scratch/links.csv
 - /Users/wsp/Documents/GitHub/Lasso/tests/scratch/nodes.csv


In [44]:
model_roadway_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2020-07-21 10:12:11, INFO: Starting fixed width convertion
2020-07-21 10:14:13, INFO: Writing out link database
2020-07-21 10:14:14, INFO: Writing out link header and width ----
2020-07-21 10:14:14, INFO: Starting fixed width convertion
2020-07-21 10:14:18, INFO: Writing out node database
2020-07-21 10:14:18, INFO: Writing out node header and width


In [45]:
standard_transit_net = StandardTransit.fromTransitNetwork(transit_net)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin"))

2020-07-21 10:14:18, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2020-07-21 10:14:18, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard


## Make Cube .net File

In [49]:
os.environ['LINK_DATA_PATH'] = os.path.join(output_dir, 'links.txt')
os.environ['NODE_DATA_PATH'] = os.path.join(output_dir, 'nodes.txt')
command = 'call runtpp ' + output_dir + 'make_complete_network_from_fixed_width_file.s'
os.system(command)

32512